<h1 align=center><font size = 10>The "Battle" of Neighborhoods : Manhattan vs Staten Island </font></h1>

# -Let's make the same steps for Staten Island
## I don't provide more comments, because all steps are the same , only one . We will be forced to correct a list of Basic necessities , because the list of Manhattan places doesn't match a list is Staten Island

KeyError: "['Pet Service', 'Baby Store', 'Veterinarian', 'Drugstore', 'Physical Therapist', 'Waterfront', 'Fish Market', 'Metro Station', 'Laundry Service', 'Health Food Store', 'Heliport', 'Medical Center', 'Kitchen Supply Store', 'Doctors Office'] not in index"

I added several new categories : Home Service, IT service , Food Service, Insurance office and so on 

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## - Download and Explore Dataset

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [5]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [6]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [13]:
neighborhoods.shape

(306, 4)

In [14]:
neighborhoods.dtypes

Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [15]:
neighborhoods.values

array([['Bronx', 'Wakefield', 40.89470517661, -73.84720052054902],
       ['Bronx', 'Co-op City', 40.87429419303012, -73.82993910812398],
       ['Bronx', 'Eastchester', 40.887555677350775, -73.82780644716412],
       ...,
       ['Queens', 'Bayswater', 40.611321691283834, -73.76596781445627],
       ['Queens', 'Queensbridge', 40.756091297094706, -73.94563070334091],
       ['Staten Island', 'Fox Hills', 40.61731079252983,
        -74.08173992211962]], dtype=object)

In [16]:
NG=neighborhoods.groupby('Borough').count()[['Neighborhood']]
NG.sort_values(by='Neighborhood', ascending=False)
#NG.head()

,Neighborhood
Borough,
Queens,81
Brooklyn,70
Staten Island,63
Bronx,52
Manhattan,40


#### Use geopy library to get the latitude and longitude values of New York City.

In [17]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [18]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Let's have a look at Staten Island

In [19]:
StIs_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)
StIs_data.head(63)

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182
5,Staten Island,Grymes Hill,40.624185,-74.087248
6,Staten Island,Todt Hill,40.597069,-74.111329
7,Staten Island,South Beach,40.580247,-74.079553
8,Staten Island,Port Richmond,40.633669,-74.129434
9,Staten Island,Mariner's Harbor,40.632546,-74.150085


In [20]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Staten Island are 40.5834557, -74.1496048.


In [21]:
# create map of Staten Island using latitude and longitude values
map_StIs = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(StIs_data['Latitude'], StIs_data['Longitude'], StIs_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_StIs)  
    
map_StIs

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = '0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD' # your Foursquare ID
CLIENT_SECRET = 'ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1' # your Foursquare Secret
VERSION = '20181102'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD
CLIENT_SECRET:ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

## Let's investigate Staten Island

In [23]:
StIs_data.head(63)

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182
5,Staten Island,Grymes Hill,40.624185,-74.087248
6,Staten Island,Todt Hill,40.597069,-74.111329
7,Staten Island,South Beach,40.580247,-74.079553
8,Staten Island,Port Richmond,40.633669,-74.129434
9,Staten Island,Mariner's Harbor,40.632546,-74.150085


In [24]:
StIs_data.loc[0, 'Neighborhood']

'St. George'

In [25]:
neighborhood_latitude = StIs_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = StIs_data.loc[0, 'Longitude'] # neighborhood longitude valum

neighborhood_name = StIs_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. George are 40.6449815710044, -74.07935312512797.


In [26]:
# type your answer here
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0RJKNKVNGQ4TBXPY0DQN4MRJIXZO1YS3W2MSOLGESJT34MBD&client_secret=ZFY50IOKW3W5PBLSL41OW10HTKG3K2HVX1XWIRJTHPRLVFK1&v=20181102&ll=40.6449815710044,-74.07935312512797&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e98a6deedbcad001b773050'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 40.6494815755044,
    'lng': -74.07343346476772},
   'sw': {'lat': 40.6404815665044, 'lng': -74.08527278548821}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6924c46d4a5938e7ac6c0',
       'name': 'A&S Pizzeria',
       'location': {'address': '87 Stuyvesant Pl',
        'crossStreet': 'Wall st',
        'lat': 40.64393953223924,
        'lng': -74.0776259226109,
 

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(105)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,A&S Pizzeria,Pizza Place,40.643940,-74.077626
1,Beso,Tapas Restaurant,40.643306,-74.076508
2,Richmond County Bank Ballpark,Baseball Stadium,40.645056,-74.076864
3,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510
4,Nike Factory Store,Sporting Goods Shop,40.645753,-74.077702
5,Shake Shack,Burger Joint,40.643660,-74.075891
6,Enoteca Maria,Italian Restaurant,40.641941,-74.077320
7,St. George Theatre,Theater,40.642253,-74.077496
8,Marie's 2,Italian Restaurant,40.642176,-74.076669
9,Steiny's Pub,Bar,40.642185,-74.076599


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

32 venues were returned by Foursquare.


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<a id='item5'></a>

In [32]:
# type your answer here

StIs_venues = getNearbyVenues(names=StIs_data['Neighborhood'],
                                   latitudes=StIs_data['Latitude'],
                                   longitudes=StIs_data['Longitude']
                                  )


St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [33]:
print(StIs_venues.shape)
StIs_venues.head()

(829, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
3,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
4,St. George,40.644982,-74.079353,Nike Factory Store,40.645753,-74.077702,Sporting Goods Shop


In [34]:
print(StIs_venues.shape)
StIs_venues.tail()

(829, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
824,Richmond Valley,40.519541,-74.229571,peri's pearl tea,40.516871,-74.233268,Bubble Tea Shop
825,Richmond Valley,40.519541,-74.229571,Mizu Hibachi Japanese Fusion,40.518215,-74.234651,Food
826,Fox Hills,40.617311,-74.081740,SUBWAY,40.618939,-74.082881,Sandwich Place
827,Fox Hills,40.617311,-74.081740,MTA S76,40.616170,-74.083185,Bus Station
828,Fox Hills,40.617311,-74.081740,MTA Bus - Tompkins Av & Vanderbilt Av (S52/S76...,40.620052,-74.077180,Bus Stop


In [35]:
StIs_venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Annadale             12
Arden Heights         7
Arlington             8
Arrochar             20
Bay Terrace          11
Bloomfield            5
Bulls Head           40
Butler Manor          6
Castleton Corners    14
Charleston           29
Chelsea               5
Clifton              21
Concord               8
Dongan Hills         22
Egbertville           4
Elm Park              9
Eltingville          41
Emerson Hill          1
Fox Hills             3
Graniteville          4
Grant City           22
Grasmere             19
Great Kills          21
Greenridge            6
Grymes Hill           2
Heartland Village    10
Huguenot              8
Lighthouse Hill       5
Manor Heights        14
Mariner's Harbor      6
Midland Beach        11
New Brighton         13
New Dorp             26
New Dorp Beach       13
New Springville      29
Oakwood               4
Old Town             20
Park Hill             6
Pleasant Plains      20
Port Ivory            1
Port Richmond         4
Pri

In [36]:
print('There are {} uniques categories.'.format(len(StIs_venues['Venue Category'].unique())))

There are 184 uniques categories.


In [37]:
# one hot encoding
StIs_onehot = pd.get_dummies(StIs_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
StIs_onehot['Neighborhood'] = StIs_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [StIs_onehot.columns[-1]] + list(StIs_onehot.columns[:-1])
StIs_onehot = StIs_onehot[fixed_columns]

StIs_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Service,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lawyer,Liquor Store,Locksmith,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [38]:
StIs_onehot.shape

(829, 185)

In [39]:
StIs_onehot.columns = StIs_onehot.columns.str.replace("'","")
StIs_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Big Box Store,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Service,Food Truck,French Restaurant,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lawyer,Liquor Store,Locksmith,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Mens Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toll Plaza,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Womens Store,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [40]:
Basic_necessities=StIs_onehot[['Neighborhood','Baby Store','Bakery','Bank','Boat or Ferry','Bus Station','Bus Stop','Clothing Store','Doctors Office','Drugstore','Dry Cleaner','Electronics Store','Farmers Market','Fish Market','Heliport','Health & Beauty Service','Gas Station','Grocery Store','Health Food Store','Kids Store','Kitchen Supply Store','Laundry Service','Market','Medical Center','Metro Station','Optical Shop','Pet Service','Pet Store','Pharmacy','Physical Therapist','Trail','Train Station','Veterinarian','Waterfront']]
Basic_necessities.head()

KeyError: "['Pet Service', 'Baby Store', 'Veterinarian', 'Drugstore', 'Physical Therapist', 'Waterfront', 'Fish Market', 'Metro Station', 'Laundry Service', 'Health Food Store', 'Heliport', 'Medical Center', 'Kitchen Supply Store', 'Doctors Office'] not in index"

# - Corrected list of Basic necessities

In [41]:
Basic_necessities=StIs_onehot[['Neighborhood','Bakery','Bank','Beach','Boat or Ferry','Bus Station','Bus Stop','Business Service','Clothing Store','Electronics Store','Dry Cleaner','Farmers Market','Food Service','Grocery Store','Harbor / Marina','Hardware Store','Home Service','IT Services','Insurance Office','Lawyer','Kids Store','Market','Optical Shop','Other Repair Shop','Pet Store','Pharmacy','Trail','Train Station']]
Basic_necessities.head()

,Neighborhood,Bakery,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store,Dry Cleaner,Farmers Market,Food Service,Grocery Store,Harbor / Marina,Hardware Store,Home Service,IT Services,Insurance Office,Lawyer,Kids Store,Market,Optical Shop,Other Repair Shop,Pet Store,Pharmacy,Trail,Train Station
0,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
StIs_grouped = Basic_necessities.groupby('Neighborhood').mean().reset_index()
StIs_grouped

,Neighborhood,Bakery,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store,Dry Cleaner,Farmers Market,Food Service,Grocery Store,Harbor / Marina,Hardware Store,Home Service,IT Services,Insurance Office,Lawyer,Kids Store,Market,Optical Shop,Other Repair Shop,Pet Store,Pharmacy,Trail,Train Station
0,Annadale,0.083333,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.083333
1,Arden Heights,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.0,0.000000
2,Arlington,0.000000,0.000000,0.000000,0.125,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.125,0.000000,0.00000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000
3,Arrochar,0.000000,0.000000,0.000000,0.000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000
4,Bay Terrace,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.090909
5,Bloomfield,0.000000,0.000000,0.000000,0.000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000
6,Bulls Head,0.000000,0.025000,0.000000,0.000,0.000000,0.075000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.025000,0.050000,0.0,0.000000
7,Butler Manor,0.000000,0.000000,0.000000,0.000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000
8,Castleton Corners,0.000000,0.071429,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000,0.071429,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000
9,Charleston,0.034483,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00000,0.000,0.034483,0.00000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00,0.034483,0.000000,0.0,0.000000


In [43]:
StIs_grouped.shape

(62, 28)

In [53]:
for col in StIs_grouped.columns: 
    print(col) 

Neighborhood
Bakery
Bank
Beach
Boat or Ferry
Bus Station
Bus Stop
Business Service
Clothing Store
Electronics Store
Dry Cleaner
Farmers Market
Food Service
Grocery Store
Harbor / Marina
Hardware Store
Home Service
IT Services
Insurance Office
Lawyer
Kids Store
Market
Optical Shop
Other Repair Shop
Pet Store
Pharmacy
Trail
Train Station


In [44]:
num_top_venues = 10

for hood in StIs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = StIs_grouped[StIs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annadale----
               venue  freq
0             Bakery  0.08
1      Train Station  0.08
2              Beach  0.00
3       Home Service  0.00
4              Trail  0.00
5           Pharmacy  0.00
6          Pet Store  0.00
7  Other Repair Shop  0.00
8       Optical Shop  0.00
9             Market  0.00


----Arden Heights----
               venue  freq
0           Pharmacy  0.14
1           Bus Stop  0.14
2   Business Service  0.14
3             Lawyer  0.14
4             Bakery  0.00
5       Home Service  0.00
6              Trail  0.00
7          Pet Store  0.00
8  Other Repair Shop  0.00
9       Optical Shop  0.00


----Arlington----
               venue  freq
0           Bus Stop  0.25
1       Home Service  0.12
2      Boat or Ferry  0.12
3       Food Service  0.12
4             Bakery  0.00
5              Trail  0.00
6           Pharmacy  0.00
7          Pet Store  0.00
8  Other Repair Shop  0.00
9       Optical Shop  0.00


----Arrochar----
               venue  freq
0 

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = StIs_grouped['Neighborhood']

for ind in np.arange(StIs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(StIs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annadale,Train Station,Bakery,Pharmacy,Food Service,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service
1,Arden Heights,Pharmacy,Bus Stop,Lawyer,Business Service,Train Station,Food Service,Bank,Beach,Boat or Ferry,Bus Station
2,Arlington,Bus Stop,Boat or Ferry,Home Service,Food Service,Train Station,Grocery Store,Bank,Beach,Bus Station,Business Service
3,Arrochar,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
4,Bay Terrace,Train Station,Insurance Office,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store


In [47]:
# set number of clusters
kclusters = 5

StIs_grouped_clustering = StIs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(StIs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 0, 2, 0, 2, 0, 0], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

StIs_merged = StIs_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
StIs_merged = StIs_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#StIs_merged.astype({'Cluster Labels': 'int32'}).dtypes

StIs_merged.head() # check the last columns!


#df.astype({'col1': 'int32'}).dtypes
#col1    int32
#col2    int64
#dtype: object

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,0.0,Clothing Store,Harbor / Marina,Pharmacy,Farmers Market,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop
1,Staten Island,New Brighton,40.640615,-74.087017,1.0,Bus Stop,Home Service,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store
2,Staten Island,Stapleton,40.626928,-74.077902,0.0,Bank,Train Station,Pharmacy,Optical Shop,Food Service,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service
3,Staten Island,Rosebank,40.615305,-74.069805,0.0,Grocery Store,Pharmacy,Beach,Electronics Store,Train Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Business Service
4,Staten Island,West Brighton,40.631879,-74.107182,0.0,Pharmacy,Bank,Bus Stop,Train Station,Grocery Store,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store


In [49]:
StIs_merged.dtypes

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [50]:
StIs_merged['Cluster Labels'] = StIs_merged['Cluster Labels'].fillna(0).astype(int)

In [51]:
StIs_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Staten Island,St. George,40.644982,-74.079353,0,Clothing Store,Harbor / Marina,Pharmacy,Farmers Market,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop
1,Staten Island,New Brighton,40.640615,-74.087017,1,Bus Stop,Home Service,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store
2,Staten Island,Stapleton,40.626928,-74.077902,0,Bank,Train Station,Pharmacy,Optical Shop,Food Service,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service
3,Staten Island,Rosebank,40.615305,-74.069805,0,Grocery Store,Pharmacy,Beach,Electronics Store,Train Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Business Service
4,Staten Island,West Brighton,40.631879,-74.107182,0,Pharmacy,Bank,Bus Stop,Train Station,Grocery Store,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store


In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(StIs_merged['Latitude'], StIs_merged['Longitude'], StIs_merged['Neighborhood'], StIs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## -Let's have a look more in details on each cluster

#### Cluster 1

In [61]:
print('*CLUSTER 1 (red dot)*')
StIs_merged.loc[StIs_merged['Cluster Labels'] == 0, StIs_merged.columns[[1] + list(range(5, StIs_merged.shape[1]))]]


*CLUSTER 1 (red dot)*


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. George,Clothing Store,Harbor / Marina,Pharmacy,Farmers Market,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop
2,Stapleton,Bank,Train Station,Pharmacy,Optical Shop,Food Service,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service
3,Rosebank,Grocery Store,Pharmacy,Beach,Electronics Store,Train Station,Bank,Boat or Ferry,Bus Station,Bus Stop,Business Service
4,West Brighton,Pharmacy,Bank,Bus Stop,Train Station,Grocery Store,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store
5,Grymes Hill,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store
8,Port Richmond,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store
10,Port Ivory,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store
11,Castleton Corners,Grocery Store,Bank,Train Station,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store
12,New Springville,Pharmacy,Optical Shop,Bus Stop,Grocery Store,Train Station,Food Service,Bank,Beach,Boat or Ferry,Bus Station
13,Travis,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store,Electronics Store


#### Cluster 2

In [55]:
StIs_merged.loc[StIs_merged['Cluster Labels'] == 1, StIs_merged.columns[[1] + list(range(5, StIs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,New Brighton,Bus Stop,Home Service,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store
24,Park Hill,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
52,Randall Manor,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
56,Willowbrook,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
62,Fox Hills,Bus Station,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Business Service,Clothing Store,Electronics Store


#### Cluster 3

In [56]:
StIs_merged.loc[StIs_merged['Cluster Labels'] == 2, StIs_merged.columns[[1] + list(range(5, StIs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,South Beach,Beach,Bus Stop,Train Station,Grocery Store,Bank,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
9,Mariner's Harbor,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
20,Tottenville,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
27,Arlington,Bus Stop,Boat or Ferry,Home Service,Food Service,Train Station,Grocery Store,Bank,Beach,Bus Station,Business Service
28,Arrochar,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
29,Grasmere,Bus Stop,Bank,Bakery,Pharmacy,IT Services,Grocery Store,Food Service,Beach,Boat or Ferry,Bus Station
32,Midland Beach,Beach,Bus Stop,Train Station,Grocery Store,Bank,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
38,Butler Manor,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store
41,Arden Heights,Pharmacy,Bus Stop,Lawyer,Business Service,Train Station,Food Service,Bank,Beach,Boat or Ferry,Bus Station
45,Bloomfield,Bus Stop,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Business Service,Clothing Store,Electronics Store


#### Cluster 4

In [57]:
StIs_merged.loc[StIs_merged['Cluster Labels'] == 3, StIs_merged.columns[[1] + list(range(5, StIs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Oakwood,Bus Station,Lawyer,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Stop,Business Service,Clothing Store
26,Graniteville,Grocery Store,Bus Station,Train Station,Bank,Beach,Boat or Ferry,Bus Stop,Business Service,Clothing Store,Electronics Store


#### Cluster 5

In [58]:
StIs_merged.loc[StIs_merged['Cluster Labels'] == 4, StIs_merged.columns[[1] + list(range(5, StIs_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Todt Hill,Trail,Train Station,Grocery Store,Bank,Beach,Boat or Ferry,Bus Station,Bus Stop,Business Service,Clothing Store
